<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_MLPMixer/test_sample_MLPMixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c dogs-vs-cats

In [ ]:
# ! kaggle datasets download -d dansbecker/hot-dog-not-hot-dog
# ! unzip -q hot-dog-not-hot-dog.zip

In [5]:
! unzip -q train.zip
! rm train.zip
! unzip -q test1.zip
! rm test1.zip

In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/modeling_MLPMixer

fatal: destination path 'PyTorch-Architectures' already exists and is not an empty directory.
/content/PyTorch-Architectures/modeling_MLPMixer


In [7]:
! mv /content/train .
! mv /content/test1 .

In [2]:
import os
import time
from PIL import Image
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from model import MLPMixer

In [3]:
samples = os.listdir('train')
print('Total Samples: ', len(samples))

Total Samples:  25000


In [4]:
parent_path = '/content/PyTorch-Architectures/modeling_MLPMixer/train/'
samples = [parent_path + str(sample) for sample in samples]
samples = [(sample, 1) if 'dog' in sample else (sample, 0) for sample in samples]

In [5]:
random.shuffle(samples)

split = 90 * len(samples) // 100
train_samples = samples[:split]
valid_samples = samples[split:]

print('Train Samples: ', len(train_samples))
print('Valid Samples: ', len(valid_samples))

Train Samples:  22500
Valid Samples:  2500


In [6]:
class CustomDataset(Dataset):
  def __init__(self, path_images, transforms=None):
    self.path_images = path_images
    self.transforms = transforms
  
  def __len__(self):
    return len(self.path_images)
  
  def __getitem__(self, idx):
    images, labels = self.path_images[idx][0], self.path_images[idx][1]
    images = Image.open(images)
    if self.transforms is not None:
      images = self.transforms(images)
    return {
        'img': images,
        'labels': torch.tensor(labels),
    }

transformations = transforms.Compose([
                                 transforms.Resize(size=(256, 256)),
                                 transforms.ToTensor(),
])

In [9]:
sample_dataset = CustomDataset(path_images=train_samples, transforms=transformations)
sample_loader = DataLoader(dataset=sample_dataset, batch_size=2)
for sample in sample_loader:
  print(sample['img'].shape, sample['labels'].shape)
  print(sample['labels'])
  break

torch.Size([2, 3, 256, 256]) torch.Size([2])
tensor([1, 0])


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = MLPMixer(image_size=256,
                 patch_size=16,
                 channel=3,
                 dim=512,
                 depth=8,
                 num_classes=2)
model.to(device)

In [11]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Trainable Parameters: ', params)

Trainable Parameters:  988418


In [12]:
# Hyperparameter section
EPOCHS = 3
LR = 3e-4
BATCH_SIZE = 32

In [13]:
train_dataset = CustomDataset(path_images=train_samples, transforms=transformations)
valid_dataset = CustomDataset(path_images=valid_samples, transforms=transformations)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

print('Length of Train Loader: ', len(train_loader))
print('Length of Valid Loader: ', len(valid_loader))

Length of Train Loader:  704
Length of Valid Loader:  79
